In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

model_id = "nomic-ai/nomic-embed-text-v2-moe"
prompt = "Cactus activation inspection test."

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model = AutoModel.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

print("Model loaded successfully.")
print(next(model.parameters()).dtype)

config.json: 0.00B [00:00, ?B/s]

configuration_hf_nomic_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


modeling_hf_nomic_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/7710840340a098cfb869c4f65e87cf2b1b70caca/modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


model.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

Model loaded successfully.
torch.float32


In [4]:
def hook_fn(module, input_tensors, output_tensors):
    module_name = module_map.get(module, "Unknown Module")
    print(f"{module_name:<50}")

    if hasattr(module, 'weight') and isinstance(module.weight, torch.Tensor):
        w = module.weight.data
        # print(f"  -> Weight Shape:     {w.shape}")
        # print(f"  -> Weight Precision:   {w.dtype}")
        # print(f"  -> Weight FIrst 5:    {w.flatten().detach().cpu()[:5].tolist()}")

    if hasattr(module, 'bias') and module.bias is not None:
        b = module.bias.data
        print(f"  -> Bias Shape:       {b.shape}")
        print(f"  -> Bias Precision:     {b.dtype}")

    if isinstance(output_tensors, torch.Tensor):
        print(f"  | Output Shape:       {output_tensors.shape}")
        print(f"  | Elements:   {output_tensors.flatten().detach().cpu()[:5]}")
    elif isinstance(output_tensors, (list, tuple)):
        for i, t in enumerate(output_tensors):
             if isinstance(t, torch.Tensor):
                print(f"  | Output {i} Shape:     {t.shape}")
                print(f"  | Output {i} First 5:   {t.flatten().detach().cpu()[:5]}")
             else:
                print(f"  | Output {i} Type:      {type(t)}")
    else:
        print(f"  | Output Type:        {type(output_tensors)}")
    print("-" * 60)

    # if module_name == 'model.layers.0':
    #     raise StopForward()

hook_handles = []
module_map = {module: name for name, module in model.named_modules()}

print("Registering forward hooks on all modules...")
for name, module in model.named_modules():
    handle = module.register_forward_hook(hook_fn)
    hook_handles.append(handle)
print(f"Registered {len(hook_handles)} hooks.\n")

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(f"Running model forward pass to trigger hooks...\n")
print(inputs)

try:
    model.eval()
    with torch.no_grad():
        results = model(**inputs).last_hidden_state[0]
# except StopForward:
#     print("\nInspection stopped after the first transformer block as requested.")
finally:
    for handle in hook_handles:
        handle.remove()
    print("=" * 80)
    print(f"Removed all {len(hook_handles)} hooks.")

print("\nScript finished.")

Registering forward hooks on all modules...
Registered 176 hooks.

Running model forward pass to trigger hooks...

{'input_ids': tensor([[     0,   2041,  24392,  34704,   1363, 134071,   1830,   3034,      5,
              2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
embeddings.word_embeddings                        
  | Output Shape:       torch.Size([1, 10, 768])
  | Elements:   tensor([ 0.1003,  0.1938,  0.1553,  0.2286, -0.0408])
------------------------------------------------------------
embeddings.token_type_embeddings                  
  | Output Shape:       torch.Size([10, 768])
  | Elements:   tensor([-0.0029, -0.0315, -0.0494, -0.0114, -0.0344])
------------------------------------------------------------
embeddings                                        
  | Output Shape:       torch.Size([1, 10, 768])
  | Elements:   tensor([ 0.0974,  0.1624,  0.1059,  0.2172, -0.0752])
------------------------------------------------------------
emb_ln              

In [6]:
tokens = tokenizer(prompt, return_tensors="pt")
print(tokens)
print(tokenizer.convert_ids_to_tokens(tokens['input_ids'][0]))

{'input_ids': tensor([[     0,   2041,  24392,  34704,   1363, 134071,   1830,   3034,      5,
              2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['<s>', '▁Ca', 'ctus', '▁activa', 'tion', '▁inspect', 'ion', '▁test', '.', '</s>']


In [5]:
model.config

NomicBertConfig {
  "activation_function": "gelu",
  "add_pooling_layer": false,
  "architectures": [
    "NomicBertModel"
  ],
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "nomic-ai/nomic-bert-2048--configuration_hf_nomic_bert.NomicBertConfig",
    "AutoModel": "nomic-ai/nomic-bert-2048--modeling_hf_nomic_bert.NomicBertModel",
    "AutoModelForMaskedLM": "nomic-ai/nomic-bert-2048--modeling_hf_nomic_bert.NomicBertForPreTraining",
    "AutoModelForMultipleChoice": "nomic-ai/nomic-bert-2048--modeling_hf_nomic_bert.NomicBertForMultipleChoice",
    "AutoModelForQuestionAnswering": "nomic-ai/nomic-bert-2048--modeling_hf_nomic_bert.NomicBertForQuestionAnswering",
    "AutoModelForSequenceClassification": "nomic-ai/nomic-bert-2048--modeling_hf_nomic_bert.NomicBertForSequenceClassification",
    "AutoModelForTokenClassification": "nomic-ai/nomic-bert-2048--modeling_hf_nomic_bert.NomicBertForTokenClassification"
  },
  "bos_token_id": null,
  "causal": false,
  "dense_seq_output": tru